In [10]:
# 3. Model Fine-Tuning
# Objective: Train NER models like XLM-Roberta.
# Libraries/Tools: transformers, datasets, torch, huggingface.

# Ensure transformers and datasets are installed
# %pip install transformers datasets

from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_from_disk

# Load dataset
dataset_path = r"C:\Users\Almazt\OneDrive - Ethiopian Airlines\Desktop\10 Academy\EthioMart-Week 5\telegram_data.csv"
try:
    dataset = load_dataset('csv', data_files=dataset_path, split='train')
except FileNotFoundError:
    print(f"File not found at {dataset_path}. Please check the file path and ensure the file exists.")
    # Optionally, you can load a sample dataset or handle the error as needed
    dataset = load_from_disk('path_to_sample_dataset')

import os
import requests
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_from_disk

# Disable SSL verification
os.environ['TRANSFORMERS_VERBOSITY'] = 'error'
os.environ['CURL_CA_BUNDLE'] = ''
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

# Load pre-trained model and tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=False, trust_remote_code=True, local_files_only=True)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), use_auth_token=False, trust_remote_code=True, local_files_only=True)

# Tokenize dataset
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding=True)
    labels = []
    for i, label in enumerate(examples[f"{label_column}_labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        labels.append([label[word_idx] if word_idx is not None else -100 for word_idx in word_ids])
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels)

# Training
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

c:\Users\Almazt\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\models\auto\tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like xlm-roberta-base is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.